In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\InvitadoLaptop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\InvitadoLaptop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import spotipy
import pandas as pd
import tqdm
import requests
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
CLIENT_ID = 'e89a20fa9f524fbe91a34ed52c346ce5'
CLIENT_SECRET = '5b776dd228714f9cad9716a07a6d6a25'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
artistas_list = ['Grupo 5', 'Ariana Grande', 'Linkin Park', 'Amelie lens', 'Maneskin']

album_data = []

In [6]:
for artista_nombre in tqdm.tqdm(artistas_list, desc='Obteniendo álbumes de artistas:'):
    resultados = sp.search(q=f'artist:{artista_nombre}', type='album')

    for album in resultados['albums']['items']:
        nombre_album = album['name']
        fecha_lanzamiento = album['release_date']
        id_album = album['id']

        artistas = [artist['name'] for artist in album['artists']]
        artista_principal = artistas[0] if artistas else "Artista no disponible"

        genero = "Género no disponible"  
        if album['artists']:
           artista_id = album['artists'][0]['id']
           artista_info = sp.artist(artista_id)
           generos_artista = artista_info.get('genres', [])
           if generos_artista:
              genero = generos_artista[0]  

    año_lanzamiento = fecha_lanzamiento[:10]  

    album_info = {
                 "Artista": artista_principal,
                 "Género": genero,
                 "Tipo": "Álbum",  
                 "Nombre": nombre_album,
                 "Año de lanzamiento": año_lanzamiento,
                 "ID": id_album,
                 "Todos los artistas": ", ".join(artistas)
                 }

    album_data.append(album_info)

    
df_albums = pd.DataFrame(album_data)

df_albums #Cambiar csv antes 

Obteniendo álbumes de artistas:: 100%|██████████| 5/5 [00:04<00:00,  1.14it/s]


,Artista,Género,Tipo,Nombre,Año de lanzamiento,ID,Todos los artistas
0,Grupo 5,cumbia,Álbum,Teatro Gran Rex (En Vivo),2018-07-01,4ZtxnbQnTZCkmdELHPmebJ,Grupo 5
1,Ariana Grande,pop,Álbum,Santa Tell Me,2014-11-24,27MNgBEnLCKoafz1g2Zu97,Ariana Grande
2,Linkin Park,nu metal,Álbum,Meteora 20th Anniversary Edition,2023-04-07,3Q9wXhEAX7NYCPP0hxIuDz,Linkin Park
3,Amelie Lens,techno,Álbum,Hypnotized,2019-07-05,68XbUQdwTqfl0Hl3ZHJWgW,Amelie Lens
4,Måneskin,Género no disponible,Álbum,MAMMAMIA,2021-10-08,7lVXNBav3Z8oA4jGk62Gme,Måneskin


In [7]:

API_KEY = "d43d3344b1962fd39c8e7dd314d2296c"  # Reemplaza con tu clave de Last.fm
ARTISTS = ['Grupo 5', 'Ariana Grande', 'Linkin Park', 'Amelie lens', 'Maneskin']  # Lista de artistas

artist_data = []

def get_artist_info(artist):
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist}&api_key={API_KEY}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "artist" in data:
            artist_info = {
                "Artista": data['artist']['name'],
                "Biografía": data['artist']['bio']['summary'],  
                "Oyentes": data['artist']['stats']['listeners'],  
                "Reproducciones": data['artist']['stats']['playcount'],  
                "URL": data['artist']['url'], 
                "Todos los artistas": artist 
            }
            artist_data.append(artist_info)
        else:
            print(f"No se encontró el artista: {artist}")
    else:
        print(f"Error en la solicitud para {artist}: {response.status_code}")

# Llama a la función para cada artista en la lista
for artist in ARTISTS:
    get_artist_info(artist)

# Crea el DataFrame después de recopilar todos los datos
df_artist = pd.DataFrame(artist_data)

df_artist #Cambiar csv antes 

,Artista,Biografía,Oyentes,Reproducciones,URL,Todos los artistas
0,Grupo 5,Successful and one of the most popular dancing...,40673,1057160,https://www.last.fm/music/Grupo+5,Grupo 5
1,Ariana Grande,"Ariana Grande-Butera, better known professiona...",3727621,801551412,https://www.last.fm/music/Ariana+Grande,Ariana Grande
2,Linkin Park,Linkin Park is an American nu-metal band from ...,5855856,521899308,https://www.last.fm/music/Linkin+Park,Linkin Park
3,Amelie Lens,Amelie Lens (born 31 May 1990) is a Belgian el...,120816,1151676,https://www.last.fm/music/Amelie+Lens,Amelie lens
4,Maneskin,Måneskin is an Italian rock band formed in Rom...,5103,82410,https://www.last.fm/music/Maneskin,Maneskin


In [8]:
similar_artist_data = []

def get_similar_artists(artist):
    url_similar = f"http://ws.audioscrobbler.com/2.0/?method=artist.getsimilar&artist={artist}&api_key={API_KEY}&format=json" # Corrected method
    response_similar = requests.get(url_similar)

    if response_similar.status_code == 200:
        data_similar = response_similar.json()

        if 'similarartists' in data_similar and 'artist' in data_similar['similarartists']: # added check
            for similar_artist in data_similar['similarartists']['artist']:
                similar_artist_data.append(similar_artist)
        else:
            print(f"No hay artistas similares para {artist} o la estructura de la respuesta es incorrecta.")
    else:
        print(f"Error en la solicitud para artistas similares de {artist}: {response_similar.status_code}")

# Llama a la función para cada artista en la lista
for artist in ARTISTS: # corrected variable name
    get_similar_artists(artist)

# Crea el DataFrame después de recopilar todos los datos
df_similar_artist = pd.DataFrame(similar_artist_data)

df_similar_artist #Cambiar csv antes 


,name,mbid,match,url,image,streamable
0,Agua Marina,0fbbb422-93a0-4c95-a378-6c72546473b7,1,https://www.last.fm/music/Agua+Marina,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
1,Armonia 10,NaN,0.931969,https://www.last.fm/music/Armonia+10,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
2,Bareto,83dce32c-bb29-48f3-937e-e1d10762ebb6,0.529827,https://www.last.fm/music/Bareto,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
3,Ráfaga,890a34a7-bfcb-4e3a-b70b-47a4eadc343f,0.188361,https://www.last.fm/music/R%C3%A1faga,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
4,Armonía 10,d873f9ee-d46f-45e3-9b92-6d2bfab2bb0d,0.180759,https://www.last.fm/music/Armon%C3%ADa+10,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
...,...,...,...,...,...,...
495,La ragazza dello Sputnik,NaN,0.082382,https://www.last.fm/music/La+ragazza+dello+Spu...,[{'#text': 'https://lastfm.freetls.fastly.net/...,0
496,picciotto,NaN,0.082382,https://www.last.fm/music/picciotto,"[{'#text': '', 'size': 'small'}, {'#text': '',...",0
497,Alice Clarini,NaN,0.082382,https://www.last.fm/music/Alice+Clarini,"[{'#text': '', 'size': 'small'}, {'#text': '',...",0
498,Poeti Maledetti,NaN,0.082382,https://www.last.fm/music/Poeti+Maledetti,"[{'#text': '', 'size': 'small'}, {'#text': '',...",0
